<h1 style="color:black" align="center">Настройки перед обработкой данных</h1>

### Подключение бибилиотек

In [1]:
import numpy as np
import pandas as pd

### Загрузка данных

In [2]:
data = pd.read_csv('./input_data/database.csv')

**Удаление признаков**

Так как сбор данных произволился в одном городе и штате, мы можем убрать параметр `city`, `state`. Если бы данные были собраны по другим регионам, то данный признак имел бы большое значение.

Удалим также некоторые признаки, которые в данный момент не имеют смысла.

In [3]:
drop_columns = ['url', 'description', 'photos', 'city', 'state']
data.drop(drop_columns, axis=1, inplace=True)

<h1 style="color:black" align="center">Выполняем обработку пропусков</h1>

In [4]:
for column in data.columns:
    null_data = data[column].isnull().sum()
    if null_data:
        print(f'{column} - {null_data}')

bathrooms - 1
bathroomsFull - 4
bedrooms - 18
fireplaceFeatures - 384
fireplaces - 113
garageSpaces - 63
hasAttachedGarage - 336
hasCarport - 56
hasHeating - 148
heating - 149
laundryFeatures - 171
livingArea - 1
parkingFeatures - 6
propertyTaxRate - 1
solarFactor - 88
specialListingConditions - 37
taxAnnualAmount - 76
taxAssessedValue - 77
yearBuilt - 1


### `yearBuilt` - год постройки

In [5]:
data.loc[data['yearBuilt'].isnull()]

,bathrooms,bathroomsFull,bedrooms,fireplaceFeatures,fireplaces,garageSpaces,hasAttachedGarage,hasCarport,hasHeating,hasPetsAllowed,...,solarClimateFactor,solarElectricityFactor,solarFactor,solarSunScore,specialListingConditions,streetAddress,taxAnnualAmount,taxAssessedValue,yearBuilt,zipcode
452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,...,6.62,5.02,0.88,84.52,Offer As Is,1 Head St,3091.0,196154.0,NaN,94112


Замечаем большое количество пропусков у других признаков, поэтому данную строку будет правильно удалить.

In [6]:
data.drop(data.loc[data['yearBuilt'].isnull()].index, axis=0, inplace=True)

### `bathroomsFull`

A full bathroom (shower and bath). Укомплектованная ванная комната (душ и ванна), гардероб.

In [7]:
data.loc[data['bathroomsFull'].isnull(), ['bathroomsFull', 'bathrooms']]

,bathroomsFull,bathrooms
341,NaN,3.0
550,NaN,5.0
551,NaN,3.0


Можем выдвинуть предположение, что `bathroomsFull` равно количеству `bathrooms`.

In [8]:
data.loc[data['bathroomsFull'].isnull(), 'bathroomsFull'] = data.loc[data['bathroomsFull'].isnull(), 'bathrooms']

### `bedrooms` - количество спален

Так как у нас нет информации о количестве комнат в доме, мы можем произвести замену комнат медианой значеним.

In [9]:
data.bedrooms.fillna(data.bedrooms.median(), inplace=True)

### `fireplaces ` - количество камионов

In [10]:
data.loc[(data['fireplaces'].isnull()) & (data['fireplaceFeatures'].notnull()), ['fireplaceFeatures', 'fireplaces']]

,fireplaceFeatures,fireplaces
84,['Wood Burning'],NaN
395,"['Wood Burning', 'Brick', 'Living Room']",NaN


Если имеются описание для камин, значит, он явно существет. Заменим число каминов часто встречающимся значением.

In [11]:
data.loc[data.index.isin([84, 395]), 'fireplaces'] = data.loc[data['fireplaces'] != 0, 'fireplaces'].mode()[0]

Все остальные значения заменим нулём, так как признак не был описан продавцом дома, значит количество равно 0.

In [12]:
data.fireplaces.fillna(0, inplace=True)

### `solarFactor` - солнечный фактор

Заменим пропуски часто встречающимся значением.

In [13]:
data.solarFactor.fillna(data['solarFactor'].mode()[0], inplace=True)

### `taxAssessedValue` - налоговая оценочная стоимость

На данный момент произведём замену средним арифметическим.

In [14]:
data.taxAssessedValue.fillna(data['taxAssessedValue'].mean(), inplace=True)

### `taxAnnualAmount` - Годовая сумма налога

На данный момент произведём замену средним арифметическим.

In [15]:
data.taxAnnualAmount.fillna(data['taxAnnualAmount'].mean(), inplace=True)

### `garageSpaces` - количество гаражей

In [16]:
data.loc[(data['garageSpaces'].isnull()) & (data['hasAttachedGarage'] == True), ['garageSpaces', 'hasAttachedGarage']]

,garageSpaces,hasAttachedGarage
341,NaN,True
550,NaN,True


Если имеется пристроенный гараж, значит и количество гаражей не может быть нулевым. Заменим число гаражей часто встречающимся значением.

In [17]:
data.loc[data.index.isin([341, 550]), 'garageSpaces'] = data.loc[data['garageSpaces'] != 0, 'garageSpaces'].mode()[0]

Все остальные значения заменим нулём, так как признак не был описан продавцом дома, значит количество равно 0.

In [18]:
data.garageSpaces.fillna(0, inplace=True)

### `specialListingConditions` - перечень особых условий продажи 

In [19]:
data.specialListingConditions.fillna(data['specialListingConditions'].mode()[0], inplace=True)

### `Множествозначные признаки`

In [20]:
data['fireplaceFeatures'].fillna('No information', inplace=True)
data['heating'].fillna('No information', inplace=True)
data['laundryFeatures'].fillna('No information', inplace=True)
data['parkingFeatures'].fillna('No information', inplace=True)

### `Бинарные признаки`

Обработав все пропуски, у нас отстались только бинарные признаки.

In [21]:
data.fillna(False, inplace=True)

**Проверяем наличие пропусков**

In [22]:
for column in data.columns:
    null_data = data[column].isnull().sum()
    if null_data:
        print(f'{column} - {null_data}')

### Сохранение данных

In [23]:
data.to_csv('./input_data/database_new.csv', index=False, encoding='utf-8')